In [17]:
import json, os

In [16]:
def get_timestamp():
    return datetime.now().strftime('%y%m%d-%H%M%S')

In [18]:
def parse_opt(opt_path):
    with open(opt_path, 'r') as f:
        opt = json.load(f)

    # path
    exp_name = opt['name']
    opt['path']['exp_root'] = os.path.join(opt['path']['root_path'],
                                           'experiments', exp_name)
    assert os.path.exists(opt['path']['exp_root']) == opt['resume'] # make sure no conflict
    opt['path']['checkpoint_dir'] = os.path.join(opt['path']['exp_root'],
                                                 'checkpoint')
    opt['path']['visual_dir'] = os.path.join(opt['path']['exp_root'],
                                             'visualization')
    opt['path']['tb_logger_root'] = opt['path']['exp_root'].replace(
        'experiments', 'tb_logger')
    for k, v in opt['path'].items():
        if k == 'root_path':
            continue
        elif k == 'tb_logger_root':
            if opt['use_tb_logger']:
                os.makedirs(v)
        else:
            os.makedirs(v)
    # dataset
    for k, v in opt['datasets'].items():
        opt['datasets'][k]['phase'] = k
        opt['datasets'][k]['num_keypoints'] = opt['num_keypoints']

    # network
    opt['networks']['hourglass']['num_keypoints'] = opt['num_keypoints']
    opt['networks']['ghcu']['output_dim'] = 2 * opt['num_keypoints']
    opt['networks']['ghcu']['in_channel'] = opt['num_keypoints']
    return opt

In [19]:
parse_opt('./options/train/train_hg.json')

{'name': 'HG',
 'num_keypoints': 5,
 'use_tb_logger': True,
 'is_train': True,
 'use_gpu': True,
 'resume': False,
 'datasets': {'train': {'type': 'celebA',
   'image_root': '/opt/data4/jzy/dataset/Face_dataset/CelebA/img_align_celeba',
   'annotation_path': '/opt/data4/Face_dataset/CelebA/CelebA/Anno/list_landmarks_align_celeba.txt',
   'partition_path': '/opt/data4/Face_dataset/CelebA/CelebA/Eval/list_eval_partition.txt',
   'input_length': 128,
   'num_workers': 4,
   'batch_size': 16,
   'flip_h': True,
   'flip_v': True,
   'rotate': True,
   'gt_sigma': 1,
   'phase': 'train',
   'num_keypoints': 5}},
 'networks': {'hourglass': {'num_feature': 256,
   'num_stack': 4,
   'num_layer': 4,
   'num_keypoints': 5},
  'ghcu': {'num_layer': 4,
   'feature_length': 256,
   'drop_rate': 0.5,
   'output_dim': 10,
   'in_channel': 5}},
 'train': {'optimizer': 'ADAM',
  'learning_rate': 0.0001,
  'weight_decay': 0,
  'lr_scheme': 'MultiStepLR',
  'lr_steps': [20, 40, 60, 80],
  'lr_gamma': 0.